**NoSQL databases** (aka "not only SQL") are non tabular, and store data differently than relational tables. NoSQL databases come in a variety of types based on their data model. The main types are document, key-value, wide-column, and graph. They provide flexible schemas and scale easily with large amounts of data and high user loads.

**Amazon DynamoDB** is a key-value and document database that delivers single-digit millisecond performance at any scale. It's a fully managed, multi-region, multi-active, durable database with built-in security, backup and restore, and in-memory caching for internet-scale applications. 

- DynamoDB can handle more than 10 trillion requests per day and can support peaks of more than 20 million requests per second.

- AWS automatically manages the data traffic of tables over multiple servers and maintains performance.

- It also relieves the customers from the burden of operating and scaling a distributed database. Hence, hardware provisioning, setup, configuration, replication, software patching, cluster scaling, etc. is managed by Amazon.

**Boto 3** Boto3 is the Amazon Web Services (AWS) Software Development Kit (SDK) for Python, which allows Python developers to write software that makes use of services like Amazon S3 and Amazon EC2.

**Session** A session stores configuration state and allows you to create service clients and resources.

**Client** and **Resource** are two different abstractions within the boto3 SDK for making AWS service requests. You would typically choose to use either the Client abstraction or the Resource abstraction to make AWS service requests. 

In [58]:
import boto3
from botocore.exceptions import ClientError
from pprint import pprint

## Create Table

The CreateTable operation adds a new table to your account. In an AWS account, table names must be unique within each Region. That is, you can have two tables with same name if you create the tables in different Regions.

**Key Schema**
Specifies the attributes that make up the primary key for a table or an index. The attributes in KeySchema must also be defined in the AttributeDefinitions array.

KeyType - The role that the key attribute will assume:

HASH - partition key - serves as the simple primary key for items in the table.

RANGE - sort key - A simple primary key might not be the best choice in all situations.  For example, what if you wanted to store two different items by the same Primary Key? To do this, we can choose a composite primary key

For a simple primary key (partition key), you must provide exactly one element with a KeyType of HASH.

For a **composite primary key** (partition key and sort key), you must provide exactly two elements, in this order: The first element must have a KeyType of HASH, and the second element must have a KeyType of RANGE.

**Provisioned Throughput** : Represents the provisioned throughput settings for a specified table or index. The settings can be modified using the UpdateTable operation. 

This is the amount of read and write activity that the table can support.

**ReadCapacityUnits**
The maximum number of strongly consistent reads consumed per second before DynamoDB returns a ThrottlingException. 


**WriteCapacityUnits**
The maximum number of writes consumed per second before DynamoDB returns a ThrottlingException.

## Load Sample Data


## CRUD Operations: Create and Read

## Read an Item
You can use the get_item method to read the item from the Movies table. 
You must specify the primary key values so that you can read any item from Movies if you know its year and title.


  

## CRUD Operations: Update and Delete 

## Update an Item

To update an existing item in an Amazon DynamoDB table, you use the UpdateItem operation. You must provide the key of the item that you want to update. You must also provide an update expression, indicating the attributes that you want to modify and the values that you want to assign to them.

An update expression specifies how UpdateItem will modify the attributes of an item—for example, setting a scalar value or removing elements from a list or a map.

The following is a syntax summary for update expressions.


update-expression ::=
    [ SET action [, action] ... ]
    
    [ REMOVE action [, action] ...]
    
    [ ADD action [, action] ... ]
    
    [ DELETE action [, action] ...]
    
An update expression consists of one or more clauses. Each clause begins with a SET, REMOVE, ADD, or DELETE keyword. You can include any of these clauses in an update expression, in any order. However, each action keyword can appear only once.


## Expression Atrribute Values

Attribute values in Amazon DynamoDB are substitutes for the actual values that you want to compare—values that you might not know until runtime. An expression attribute value must begin with a colon (:) and be followed by one or more alphanumeric characters.

## Delete an Item
You can use the delete_item method to delete one item by specifying its primary key. Optionally, you can provide a ConditionExpression to prevent the item from being deleted if the condition is not met.

**ConditionExpression** We can provide a ConditionExpression to prevent the item from being deleted if the condition is not met.

## Simple Delete

## Query the Database

You can use the query method to retrieve data from a table. You must specify a partition key value. The sort key is optional.

The primary key for the Movies table is composed of the following:

year – The partition key. The attribute type is number. 

title – The sort key. The attribute type is string.

ProjectionExpression specifies the attributes you want in the scan result.

FilterExpression specifies a condition that returns only items that satisfy the condition. All other items are discarded.

## Scan

If you are doing query then you have to pass the primary key which in your case is userId. If you do not want to use primaryKey and if you want to query using some other fields then you can do **scan with filterExpression**


The scan method reads every item in the entire table and returns all the data in the table. You can provide an **optional filter_expression** so that only the items matching your criteria are returned. However, the filter is applied only after the entire table has been scanned.

**ProjectionExpression** specifies the attributes you want in the scan result.

**FilterExpression** specifies a condition that returns only items that satisfy the condition. All other items are discarded.

A Scan operation can retrieve a maximum of 1 MB of data. This limit applies **before the filter expression is evaluated**.

The scan method returns a subset of the items each time, called a page. The LastEvaluatedKey value in the response is then passed to the scan method via the ExclusiveStartKey parameter. When the last page is returned, LastEvaluatedKey is not part of the response.



## Global Secondary Index

As more items are added to the table, scans of all the data would become slow and inefficient. Another Option we can use is the Global Secondary Index

DynamoDB allows you to create secondary indexes to account for additional data access patterns on your table. Secondary indexes are a powerful way to add query flexibility to a DynamoDB table.

DynamoDB has two kinds of secondary indexes: global secondary indexes and local secondary indexes.

Global Secondary Indexes let you define a different hash key for your table. Note that it will not change the primary hash key - author will still be the table's hash key.

GSI only provides an additional hash key for you to be able to make more complex queries.


Now we will add a global secondary index to your Category attribute that will allow you to retrieve all books in a particular category.

**Creating** a global secondary index has a lot in common with creating a table. You specify a name for the index, the attributes that will be in the index, the key schema of the index, and the provisioned throughput (the maximum capacity an application can consume from a table or index). Provisioned throughput on each index is separate from the provisioned throughput on a table.


**Backfilling** For each item in the table, DynamoDB determines which set of attributes to write to the index based on its projection (KEYS_ONLY, INCLUDE, or ALL). It then writes these attributes to the index.

During the backfilling phase, the IndexStatus attribute is set to CREATING, and the Backfilling attribute is true. The IndexStatus must be set to ACTIVE for it work and allow you make to Query.

The time required for building a global secondary index depends on several factors, such as the following:

- The size of the table
- The number of items in the table that qualify for inclusion in the index
- The number of attributes projected into the index
- The provisioned write capacity of the index
- Write activity on the main table during index builds